In [1]:
import logging
import sys
import openai
import os

from llama_index import GPTTreeIndex, GPTListIndex, SimpleDirectoryReader, LLMPredictor, PromptHelper, ServiceContext
from IPython.display import Markdown, display
from langchain.chat_models import ChatOpenAI

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

/home/marc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Choose model and tune
llm_predictor = LLMPredictor(llm=ChatOpenAI(model_name="gpt-4",
                                            request_timeout=300,
                                            temperature=1.0,
                                            top_p=1.0,
                                            frequency_penalty=0.0,
                                            presence_penalty=-1.0,
                                            max_tokens=256,
                                            openai_api_key = ""))

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

In [3]:
# Load directory which includes prompt.txt file detailing travel agent role obligations and user data.
documents = SimpleDirectoryReader('prompt/').load_data()

# Embed information in chatGPT model.
davinci_index = GPTListIndex.from_documents(documents, service_context=service_context)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens


> [build_index_from_nodes] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [4]:
# Initial data retrieval request to ChatGPT for destination airport code.
query1 = "Plan me a vacation, first return only the 3 character airport code of my destination"

In [5]:
# Send request, save response
import time

start_time = time.time()
response1 = davinci_index.query(query1)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 209 tokens


> [query] Total LLM token usage: 209 tokens


INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens


> [query] Total embedding token usage: 0 tokens
Elapsed time: 1.75 seconds


In [6]:
# Query Amadeus for hotels using chatGPT response (airport code) and save
from amadeus import Client, ResponseError

amadeus = Client(
    client_id = '',
    client_secret = ''
)

with open('response/Hotels.txt', 'w') as f:
    f.write(amadeus.reference_data.locations.hotels.by_city.get(cityCode=response1.response).body)

In [7]:
# Load directory which includes Hotels.txt file listing hotel information for destination city.
documents = SimpleDirectoryReader('response/').load_data()
# Embed hotel information in chatGPT model.
davinci_index = GPTListIndex.from_documents(documents, service_context=service_context)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens


> [build_index_from_nodes] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [8]:
# Follow-up data retreival request to chatGPT for latitude longitude of hotel recommendation.
query2 = "Now choose the best hotel and only respond with a list containing the float values for latitude then longitude of the hotel, do not add any other text besides the list containing the data."

In [9]:
# Send request, save response
import time

start_time = time.time()
response2 = davinci_index.query(query2)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")


INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 19053 tokens


> [query] Total LLM token usage: 19053 tokens


INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens


> [query] Total embedding token usage: 0 tokens
Elapsed time: 24.26 seconds


In [10]:
li = list(response2.response.replace("[", "").replace("]", "").replace(" ", "").split(","))

In [11]:
li

['39.74422', '-104.98948']

In [12]:
amadeus.shopping.activities.get(latitude=li[0], longitude=li[1]).data
#amadeus.shopping.activities.get(latitude=40.41436995, longitude=-3.69170868).data

[{'type': 'activity',
  'id': '3853965',
  'self': {'href': 'https://test.api.amadeus.com/v1/shopping/activities/3853965',
   'methods': ['GET']},
  'name': "Prospect's",
  'shortDescription': 'Unspecified',
  'description': 'Unspecified',
  'geoCode': {'latitude': 39.747383, 'longitude': -104.990364},
  'price': {'amount': '0.0', 'currencyCode': 'USD'},
  'pictures': ['https://resizer.otstatic.com/v2/photos/wide-large/1/25597345.jpg'],
  'bookingLink': 'http://www.opentable.com/r/prospects-denver?lang=en-gb&ref=18595',
  'minimumDuration': ''},
 {'type': 'activity',
  'id': '3853133',
  'self': {'href': 'https://test.api.amadeus.com/v1/shopping/activities/3853133',
   'methods': ['GET']},
  'name': "Henry's Tavern - Denver",
  'shortDescription': 'Sports Bar, Burgers, American',
  'description': 'Sports Bar, Burgers, American',
  'geoCode': {'latitude': 39.74409, 'longitude': -104.99156},
  'price': {'amount': '0.0', 'currencyCode': 'USD'},
  'pictures': ['https://resizer.otstatic.com